In [272]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline

In [291]:
df = pd.read_csv('/Users/veronicadelgado/code/vdelgadobenito/psycovid/raw_data/COVIDiSTRESS_May_30_cleaned_final.csv', encoding='latin-1')
df.head()

/Users/veronicadelgado/.pyenv/versions/3.8.6/envs/psycovid/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,answered_all,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,...,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg,Scale_UCLA_TRI_avg
0,1,No,180,2020-05-30 23:47:17,SAR,29,Female,"College degree, bachelor, master",Some College or equivalent,Not employed,...,NaN,2.900000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,No,3100,2020-05-29 23:30:15,UR,20,Male,"College degree, bachelor, master",None,Student,...,NaN,2.200000,2.333333,2.000000,5.000000,5.333333,5.0,5.000000,5.0,NaN
2,3,No,127,2020-05-30 22:40:15,SAR,47,Female,"Some College, short continuing education or eq...",Some College or equivalent,Self-employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,No,1710,2020-05-29 22:47:17,BG,79,Male,"College degree, bachelor, master",College degree,Not employed,...,NaN,3.600000,4.000000,4.000000,4.666667,4.000000,5.0,5.000000,3.9,NaN
4,5,No,2239,2020-05-29 22:42:30,SAR,61,Female,"Some College, short continuing education or eq...",Up to 12 years of school,Retired,...,Informaciones no confiables,2.714286,1.000000,3.666667,4.666667,5.333333,5.0,4.666667,5.0,NaN


In [292]:
df.shape

(125306, 154)

In [293]:
# Create a copy of the df
# Drop columns that we for sure know we are NOT going to use

unfiltered_data = df.copy()
unfiltered_data.drop(columns=['ID','answered_all','Dem_state','Duration..in.seconds.','RecordedDate','experience_war','born_92','experience_war','experience_war_TXT','war_injury','loss_during_war','time_spent_in_war','time_spent_in_war_TXT','Expl_coping_txt','Expl_Distress_txt','Final_open','AD_gain','AD_loss','AD_check'],inplace=True)

In [294]:
unfiltered_data.shape

(125306, 136)

In [302]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.16433382491181772
Dem_isolation_kids 0.17551939147758144
Scale_PSS10_UCLA_1 0.07390925633661914
Scale_PSS10_UCLA_2 0.07410761256716084
Scale_PSS10_UCLA_3 0.0739782498081119
Scale_PSS10_UCLA_4 0.07453882176399058
Scale_PSS10_UCLA_5 0.07548748199701603
Scale_PSS10_UCLA_6 0.07377126939363363
Scale_PSS10_UCLA_7 0.0750390244323131
Scale_PSS10_UCLA_8 0.07523738066285478
Scale_PSS10_UCLA_9 0.07356428897915535
Scale_PSS10_UCLA_10 0.07388338378480937
Scale_SLON_1 0.07444395574068803
Scale_SLON_2 0.07499590351263012
Scale_SLON_3 0.07465956033910291
OECD_people_1 0.10614645589161126
OECD_people_2 0.10420601450587738
OECD_insititutions_1 0.12120428104490613
OECD_insititutions_2 0.11464127706915733
OECD_insititutions_3 0.11596940139539295
OECD_insititutions_4 0.11041542694022578
OECD_i

In [295]:
# Drop columns were %NaN > 90%

for i in unfiltered_data.columns:
    percent=unfiltered_data[i].isna().sum()/len(unfiltered_data)
    if(percent > 0.90):
        unfiltered_data.drop(columns=i,inplace=True)

In [296]:
unfiltered_data.shape

(125306, 126)

In [300]:
# Drop NaN of dem features that have less 10%

unfiltered_data.dropna(subset=['UserLanguage', 'Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment', 'Country','Dem_Expat','Dem_maritalstatus','Dem_dependents','Dem_riskgroup','Dem_isolation'], inplace=True)


In [301]:
unfiltered_data.shape

(115953, 126)

## Demographic features

Important! In the paper they mention the following: "As of 30th May, the participants in our data represented 176 different countries. However, there were instances in which we only had one participant per country (i.e. The Bahamas, Uganda, etc.). For computational purposes, we decided to examine the data quality for 42 countries that had over 200 participants."

Therefore, we also need to apply this filter

In [303]:
unfiltered_data.Country.nunique()

171

In [304]:
unfiltered_data = unfiltered_data.groupby('Country').filter(lambda x : len(x)>=200)

In [305]:
unfiltered_data.Country.nunique()

42

In [306]:
unfiltered_data.shape

(113324, 126)

In [307]:
# Take all demographic columns and replace NaN by "missing"

for i in unfiltered_data.loc[:, 'Dem_isolation_adults': 'Dem_isolation_kids']:
    unfiltered_data[i].replace(np.nan,'missing',inplace=True)

In [308]:
for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.07335604108573647
Scale_PSS10_UCLA_2 0.07351487769581025
Scale_PSS10_UCLA_3 0.07344428364688857
Scale_PSS10_UCLA_4 0.07400903603826198
Scale_PSS10_UCLA_5 0.07490028590589813
Scale_PSS10_UCLA_6 0.0731972044756627
Scale_PSS10_UCLA_7 0.07445024884402245
Scale_PSS10_UCLA_8 0.07460908545409622
Scale_PSS10_UCLA_9 0.07298542232889767
Scale_PSS10_UCLA_10 0.07332956831739085
Scale_SLON_1 0.07382372665984258
Scale_SLON_2 0.07437083053898556
Scale_SLON_3 0.07407963008718366
OECD_people_1 0.1051939571494123
OECD_people_2 0.10335851187744872
OECD_insititutions_1 0.1202569623380749
OECD_insititutions_2 0.1135593519466309
OECD_insititutions_3 0.11491828738837316
OECD_insititutions_4 0.10932370901133034
OECD_insititutions_5 0.11621545303730896

## Stress features (PSS10)

In [309]:
# There are 10 questions assessing the stress levels: 'Scale_PSS10_UCLA_1' to 'Scale_PSS10_UCLA_10';
# Keep rows where participants replied to at least 8 out of 10 questions

unfiltered_data.dropna(how='any', axis=0, thresh=2, subset=['Scale_PSS10_UCLA_1', 'Scale_PSS10_UCLA_2', 'Scale_PSS10_UCLA_3', 'Scale_PSS10_UCLA_4', 'Scale_PSS10_UCLA_5', 'Scale_PSS10_UCLA_6', 'Scale_PSS10_UCLA_7', 'Scale_PSS10_UCLA_8', 'Scale_PSS10_UCLA_9', 'Scale_PSS10_UCLA_10'], inplace=True)

In [310]:
unfiltered_data.shape

(105532, 126)

In [311]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005116931357313422
Scale_PSS10_UCLA_2 0.0051832619489823
Scale_PSS10_UCLA_3 0.005116931357313422
Scale_PSS10_UCLA_4 0.005732858279953
Scale_PSS10_UCLA_5 0.006670962362127128
Scale_PSS10_UCLA_6 0.0048516089906379106
Scale_PSS10_UCLA_7 0.006206648220444984
Scale_PSS10_UCLA_8 0.0064245915930712955
Scale_PSS10_UCLA_9 0.004605238221582079
Scale_PSS10_UCLA_10 0.0049558427775461475
Scale_SLON_1 0.005524390706136527
Scale_SLON_2 0.006244551415684342
Scale_SLON_3 0.005931850054959633
OECD_people_1 0.040328999734677635
OECD_people_2 0.03840541257628018
OECD_insititutions_1 0.05651366410188379
OECD_insititutions_2 0.04934048440283516
OECD_insititutions_3 0.05075237842550127
OECD_insititutions_4 0.04479210097411212
OECD_insititutions_5 0.0

## Expl_Distress 

Set of questions identifying stressors & sources of distress. Assuming that we do not want to drop all the NaN values and we do not want to introduce a bias, these are the possible options: 
- Replace missing values with mode
- Replace missing values with 99.0 (= does not apply to my current situation)

I would go fot the second one, since the mode or most common value could be affected by other variables (country, age, etc). 

In [312]:
# Take all Expl_Distress columns and replace NaN by 99.0 (= does not apply to my current situation)

for i in unfiltered_data.loc[:, 'Expl_Distress_1': 'Expl_Distress_24']:
    unfiltered_data[i].replace(np.nan,99.0,inplace=True)

In [313]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005116931357313422
Scale_PSS10_UCLA_2 0.0051832619489823
Scale_PSS10_UCLA_3 0.005116931357313422
Scale_PSS10_UCLA_4 0.005732858279953
Scale_PSS10_UCLA_5 0.006670962362127128
Scale_PSS10_UCLA_6 0.0048516089906379106
Scale_PSS10_UCLA_7 0.006206648220444984
Scale_PSS10_UCLA_8 0.0064245915930712955
Scale_PSS10_UCLA_9 0.004605238221582079
Scale_PSS10_UCLA_10 0.0049558427775461475
Scale_SLON_1 0.005524390706136527
Scale_SLON_2 0.006244551415684342
Scale_SLON_3 0.005931850054959633
OECD_people_1 0.040328999734677635
OECD_people_2 0.03840541257628018
OECD_insititutions_1 0.05651366410188379
OECD_insititutions_2 0.04934048440283516
OECD_insititutions_3 0.05075237842550127
OECD_insititutions_4 0.04479210097411212
OECD_insititutions_5 0.0

## SPS

Available social provisions in critical/distressing situations. Again, we do not want to drop NaN and we do not want to introduce a bias. In this case there is no "buffer value" as before (does not apply to my current situation). So I would go for the mode.

In [314]:
SPS_mode = unfiltered_data.loc[:, 'SPS_1': 'SPS_10'].apply(lambda x: x.mode(dropna=False)[0])
SPS_mode

SPS_1     6.0
SPS_2     5.0
SPS_3     5.0
SPS_4     5.0
SPS_5     6.0
SPS_6     5.0
SPS_7     6.0
SPS_8     6.0
SPS_9     5.0
SPS_10    6.0
dtype: float64

In [315]:
for i in unfiltered_data.loc[:, 'SPS_1': 'SPS_10']:
    unfiltered_data[i].replace(np.nan,SPS_mode.iloc[0],inplace=True)

In [323]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005116931357313422
Scale_PSS10_UCLA_2 0.0051832619489823
Scale_PSS10_UCLA_3 0.005116931357313422
Scale_PSS10_UCLA_4 0.005732858279953
Scale_PSS10_UCLA_5 0.006670962362127128
Scale_PSS10_UCLA_6 0.0048516089906379106
Scale_PSS10_UCLA_7 0.006206648220444984
Scale_PSS10_UCLA_8 0.0064245915930712955
Scale_PSS10_UCLA_9 0.004605238221582079
Scale_PSS10_UCLA_10 0.0049558427775461475
Scale_SLON_1 0.005524390706136527
Scale_SLON_2 0.006244551415684342
Scale_SLON_3 0.005931850054959633
OECD_people_1 0.040328999734677635
OECD_people_2 0.03840541257628018
OECD_insititutions_1 0.05651366410188379
OECD_insititutions_2 0.04934048440283516
OECD_insititutions_3 0.05075237842550127
OECD_insititutions_4 0.04479210097411212
OECD_insititutions_5 0.0

**We decided to impute the NaNs of SPS_avg with KNN imputer**

In [320]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

unfiltered_data['SPS_avg'] = imputer.fit_transform(unfiltered_data[['SPS_avg']])

## Expl_Coping

16 own self-reported items to capture how people reduce their discomfort on a 6- Point Likert Scale.  Again, we do not want to drop NaN and we do not want to introduce a bias. In this case there is no "buffer value" as before (does not apply to my current situation). So I would go for the mode as before.

In [324]:
Expl_coping_mode = unfiltered_data.loc[:, 'Expl_Coping_1': 'Expl_Coping_16'].apply(lambda x: x.mode(dropna=False)[0])
Expl_coping_mode

Expl_Coping_1     4.0
Expl_Coping_2     5.0
Expl_Coping_3     5.0
Expl_Coping_4     NaN
Expl_Coping_5     5.0
Expl_Coping_6     5.0
Expl_Coping_7     1.0
Expl_Coping_8     1.0
Expl_Coping_9     5.0
Expl_Coping_10    4.0
Expl_Coping_11    4.0
Expl_Coping_12    5.0
Expl_Coping_13    5.0
Expl_Coping_14    5.0
Expl_Coping_15    1.0
Expl_Coping_16    5.0
dtype: float64

**Most common value in Expl_coping_4 is NaN, so it makes no sense to keep this column. The other option is to do inplace=True**

In [325]:
unfiltered_data.drop(columns=['Expl_Coping_4'], inplace=True)

In [326]:
for i in unfiltered_data.loc[:, 'Expl_Coping_1': 'Expl_Coping_16']:
    unfiltered_data[i].replace(np.nan,Expl_coping_mode.iloc[0],inplace=True)

In [327]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005116931357313422
Scale_PSS10_UCLA_2 0.0051832619489823
Scale_PSS10_UCLA_3 0.005116931357313422
Scale_PSS10_UCLA_4 0.005732858279953
Scale_PSS10_UCLA_5 0.006670962362127128
Scale_PSS10_UCLA_6 0.0048516089906379106
Scale_PSS10_UCLA_7 0.006206648220444984
Scale_PSS10_UCLA_8 0.0064245915930712955
Scale_PSS10_UCLA_9 0.004605238221582079
Scale_PSS10_UCLA_10 0.0049558427775461475
Scale_SLON_1 0.005524390706136527
Scale_SLON_2 0.006244551415684342
Scale_SLON_3 0.005931850054959633
OECD_people_1 0.040328999734677635
OECD_people_2 0.03840541257628018
OECD_insititutions_1 0.05651366410188379
OECD_insititutions_2 0.04934048440283516
OECD_insititutions_3 0.05075237842550127
OECD_insititutions_4 0.04479210097411212
OECD_insititutions_5 0.0

## Expl_media

6 own self-reported items to capture which information sources / media channels are used on a 6-Point Likert Scale. In this case questions 1 to 3 are specific to each country, so:
- Questions 1-3: group by country and calculate the mode based on that.
- Questions 4-6: apply mode independently of the country.

In [328]:
Expl_media_456_mode = unfiltered_data.loc[:, 'Expl_media_4': 'Expl_media_6'].apply(lambda x: x.mode(dropna=True)[0])
Expl_media_456_mode

Expl_media_4    4.0
Expl_media_5    5.0
Expl_media_6    4.0
dtype: float64

In [333]:
for i in unfiltered_data.loc[:, 'Expl_media_4': 'Expl_media_6']:
    unfiltered_data[i].replace(np.nan,Expl_media_456_mode.iloc[0],inplace=True)

In [336]:
Expl_media_1_mode = unfiltered_data.groupby('Country').Expl_media_1.apply(lambda x: x.mode(dropna=True)[0])
Expl_media_1_mode

Country
Argentina                 5.0
Australia                 5.0
Austria                   5.0
Bangladesh                5.0
Belgium                   5.0
Bosnia and Herzegovina    2.0
Brazil                    5.0
Bulgaria                  4.0
Canada                    5.0
Croatia                   5.0
Czech Republic            4.0
Denmark                   5.0
Finland                   5.0
France                    5.0
Germany                   5.0
Greece                    5.0
Hungary                   1.0
Indonesia                 5.0
Ireland                   5.0
Italy                     5.0
Japan                     4.0
Korea, South              5.0
Kosovo                    5.0
Lithuania                 5.0
Malaysia                  5.0
Mexico                    5.0
Netherlands               5.0
Pakistan                  5.0
Panama                    5.0
Philippines               4.0
Poland                    5.0
Portugal                  5.0
Romania                   4.0
Se

In [339]:
for i in unfiltered_data.loc[:, ['Expl_media_1']]:
    unfiltered_data[i].replace(np.nan,Expl_media_1_mode.iloc[0],inplace=True)

In [341]:
Expl_media_2_mode = unfiltered_data.groupby('Country').Expl_media_2.apply(lambda x: x.mode(dropna=True)[0])

In [342]:
for i in unfiltered_data.loc[:, ['Expl_media_2']]:
    unfiltered_data[i].replace(np.nan,Expl_media_2_mode.iloc[0],inplace=True)

In [343]:
Expl_media_3_mode = unfiltered_data.groupby('Country').Expl_media_3.apply(lambda x: x.mode(dropna=True)[0])

In [344]:
for i in unfiltered_data.loc[:, ['Expl_media_3']]:
    unfiltered_data[i].replace(np.nan,Expl_media_3_mode.iloc[0],inplace=True)

In [345]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005116931357313422
Scale_PSS10_UCLA_2 0.0051832619489823
Scale_PSS10_UCLA_3 0.005116931357313422
Scale_PSS10_UCLA_4 0.005732858279953
Scale_PSS10_UCLA_5 0.006670962362127128
Scale_PSS10_UCLA_6 0.0048516089906379106
Scale_PSS10_UCLA_7 0.006206648220444984
Scale_PSS10_UCLA_8 0.0064245915930712955
Scale_PSS10_UCLA_9 0.004605238221582079
Scale_PSS10_UCLA_10 0.0049558427775461475
Scale_SLON_1 0.005524390706136527
Scale_SLON_2 0.006244551415684342
Scale_SLON_3 0.005931850054959633
OECD_people_1 0.040328999734677635
OECD_people_2 0.03840541257628018
OECD_insititutions_1 0.05651366410188379
OECD_insititutions_2 0.04934048440283516
OECD_insititutions_3 0.05075237842550127
OECD_insititutions_4 0.04479210097411212
OECD_insititutions_5 0.0

In [346]:
unfiltered_data.shape

(105532, 125)

## Once this is done we can dropna and see the total values we managed to keep

In [347]:
X = unfiltered_data.dropna()

## Do we need to scale the data?